#### maizeでケモインフォマティクスのWFを作成してみる
- 02_hello_maizeをベースにrdkitを利用するワークフローを作って見ましょう。
- まず必要なライブラリ軍をインポートします

In [ ]:
from pathlib import Path
from maize.core.workflow import Workflow
from maize.core.node import Node
from maize.steps.io import LoadData, LogResult, Void, FileParameter, Return
from maize.steps.mai.molecule import LoadSmiles, LoadMolecule, Gypsum
from maize.core.interface import Input, Output
from maize.utilities.chem import IsomerCollection
from rdkit import Chem
from rdkit.Chem import Descriptors
from rdkit.Chem.Draw import IPythonConsole
import pandas as pd
from pandas import DataFrame

#### 作成するワークフローの概要
- LoadDataノードは先の例では、Intを受け取りましたが、今度はSMILESを受け取り次のノードに渡します。
- 次のノードはCalcDescという名前にし、受け取った文字列（SMILESのリスト）をRDKit のMolオブジェクトに変換、記述を計算するようにしましょう。
- 最後に計算した値をPandasのデータフレームで返します。
- 最低限、MolWt, MolLogPを計算する関数を組み込んでみましょう。

#### ノードを書いてみる時間を少し取ります。
- TIPS RDKitで計算可能な記述子リストは以下のように多数あります。

In [ ]:
for desc in Descriptors._descList:
    print(desc)

In [ ]:
class CalcDesc(Node):
    out: Output[pd.DataFrame] = Output()
    inp: Input[list[str]] = Input()

    def run(self):
        res = pd.DataFrame({'smiles': self.inp.receive()})
        res['ROMol'] = res['smiles'].apply(Chem.MolFromSmiles)
        res['MolWt'] = res['ROMol'].apply(Descriptors.MolWt)
        res['MolLogP'] = res['ROMol'].apply(Descriptors.MolLogP)
        return self.out.send(res)

In [ ]:
mols = ['CCC', 'CCOC', 'CCCC']

#### ワークフロー構築
- CalcDescノードの定義が終わったらワークフローを構築しましょう
- 非常にシンプルな例ですがLoadDataの出力をCalcDesc2私最後にそれをReturnノードで受け取るというワークフローです

In [ ]:
flow = Workflow()
load = flow.add(LoadData[list[str]])
calc = flow.add(CalcDesc)
res = flow.add(Return[pd.DataFrame])
flow.connect(load.out, calc.inp)
flow.connect(calc.out, res.inp)
load.data.set(mols)
flow.check()

In [ ]:
flow.execute()

In [ ]:
res.get()

#### maize-contribに実装されているIsomerCollectionを利用したワークフローの作成
- maize-contribには[IsomerCollection](https://github.com/MolecularAI/maize-contrib/blob/6a80f85e29e1d7e67bd13a2d34a0d5a440386fe3/maize/utilities/chem/chem.py#L1862)というクラスが実装されています。
- これはMaestro, SDF, SMILESなど様々な形式の入力を受け取りMolobjectを生成します。SMILESで受け取った場合は立体異性体のEnumerationも実施します。またこの際にRDKITを利用した立体構造生成も同時に行います。

In [ ]:
from maize.utilities.chem.chem import IsomerCollection

In [ ]:
iso1 = IsomerCollection.from_smiles('CCC(O)CN')

In [ ]:
iso1

In [ ]:
iso1.molecules[0]._molecule

In [ ]:
iso1.molecules[1]._molecule

In [ ]:
print(Chem.MolToMolBlock(iso1.molecules[0]._molecule))

In [ ]:
#### SMILESを入力としてIsomerCollectionを返すノードを書いてみて下さい

In [ ]:
class Smi2Mols(Node):
    inp: Input[list[str]] = Input()
    out: Output[list[IsomerCollection]] = Output()

    def run(self) -> None:
        smiles_list = self.inp.receive()
        mols = [IsomerCollection.from_smiles(smi) for smi in smiles_list]
        self.out.send(mols)

In [ ]:
flow = Workflow()
load = flow.add(LoadData[list[str]])
emb = flow.add(Smi2Mols)
ret = flow.add(Return[list[IsomerCollection]])
flow.connect(load.out, emb.inp)
flow.connect(emb.out, ret.inp)
load.data.set(['CC', 'CCC(O)CN'])
flow.check()

In [ ]:
flow.execute()

In [ ]:
results = ret.get()

In [ ]:
results

#### Appendix XTB calculation
- このコードを利用するためには同じ環境下に追加でXTBをインストールする必要があります
- [XTB](https://xtb-docs.readthedocs.io/en/latest/)は半経験的な量子科学計算のパッケージです。
- Maizeを利用することで入力部分の作成に労を割くことなく計算を実施できます。
- 入力はSMILESとし、そのSMILESをGypSumで3次元化次いでXTBでの計算を流すというワークフローになります。
- 
```bash
conda install -c conda-forge xtb
```

In [ ]:
from maize.steps.mai.molecule.xtb import Xtb

class SetName(Node):
    inp: Input[list[IsomerCollection]] = Input()
    out: Output[list[IsomerCollection]] = Output()
    def run(self):
        res = []
        for i, iso in enumerate(self.inp.receive()):
            iso.set_tag('component', f'mol_{i}')
            res.append(iso)

        self.out.send(res)

In [ ]:
flow = Workflow(name='xtb', cleanup_temp=True)
load = flow.add(LoadData[list[str]])
# あとでGypSumの説明をしますが、このノードも配座発生に利用するノードです。
emb = flow.add(Gypsum)
setname = flow.add(SetName)
qm = flow.add(Xtb)
res = flow.add(Return[list[IsomerCollection]])
# 入力はSMILESリストとしています
load.data.set(['c1ccccc1', 'c1cnccc1'])
# xtbノードは内部で並列処理をするのですがデフォルトで100並列の設定になっておりそのまま実施すると100 coreない場合エラーになります。
qm.n_jobs.set(5)
# 計算結果を格納するフォルダを指定します。
qm.fold.set('./xtbres')

In [ ]:
flow.connect(load.out, emb.inp)
flow.connect(emb.out, setname.inp)
flow.connect(setname.out, qm.inp)
flow.connect(qm.out, res.inp)
flow.check()

In [ ]:
flow.visualize()

In [ ]:
flow.execute()

In [ ]:
res.get()[0].molecules[0]._molecule